In [1]:
import uniprot
import word2num
import splits

In [2]:
ds = uniprot.Dataset()
release, df = ds.load('sprot')

Workdir=/tmp/tmpmreqremr


In [3]:
df.head()

,dataset,name,organism,proteinexistence,sequence
ID,,,,,
Q6GZX4,Swiss-Prot,001R_FRG3G,Frog virus 3 (isolate Goorha),4,MAFSAEDVLKEYDRRRRMEALLLSLYYPNDRKLLDYKEWSPPRVQV...
Q6GZX3,Swiss-Prot,002L_FRG3G,Frog virus 3 (isolate Goorha),4,MSIIGATRLQNDKSDTYSAGPCYAGGCSAFTPRGTCGKDWDLGEQT...
Q197F8,Swiss-Prot,002R_IIV3,Invertebrate iridescent virus 3,4,MASNTVSAQGGSNRPVRDFSNIQDVAQFLLFDPIWNEQPGSIVPWK...
Q197F7,Swiss-Prot,003L_IIV3,Invertebrate iridescent virus 3,4,MYQAINPCPQSWYGSPQLEREIVCKMSGAPHYPNYYPVHPNALGGA...
Q6GZX2,Swiss-Prot,003R_FRG3G,Frog virus 3 (isolate Goorha),3,MARPLLGKTSSVRRRLESLSACSIFFFLRKFCQKMASLVFLNSPVY...


In [4]:
len(df)

561356

In [5]:
df2 = uniprot.transform(df, max_seq_len=1400)
len(df2)

554209

In [6]:
vocab, tok, ids = word2num.transform(df2, lambda seq: [t for t in seq])

554209it [01:26, 6386.16it/s]


In [7]:
ids[:10]

['Q6GZX4',
 'Q6GZX3',
 'Q197F8',
 'Q197F7',
 'Q6GZX2',
 'Q6GZX1',
 'Q197F5',
 'Q6GZX0',
 'Q91G88',
 'Q6GZW9']

In [8]:
index = [x for x in range(len(df2))]
ix_train, ix_valid, ix_test = splits.make_train_validation_test(index, train_size=0.8, validation_size=0.1)


/home/kotliaro/anaconda3/envs/env3.6/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [9]:
print("train={} valid={} test={}".format(len(ix_train), len(ix_valid), len(ix_test)))

train=443367 valid=55421 test=55421


In [10]:
X_train = tok[ix_train]
X_valid = tok[ix_valid]

In [11]:
import train_lm


In [12]:
config = {"embed_size": 400,
         "hidden_units": 1400,
         "num_layers": 3,
         "pad_index": 0,
         "tie_encoder": True}
train_lm.train_lm(vocab, X_train, X_valid, "/tmp", "AWD_LSTM", config)